# Actividad 2
### Obtención de datos y análisis con SQLite.
### Desarrollo de las preguntas del dataset Airlines


In [6]:
# Importar librerias
import pandas as pd
import sqlite3
from google.colab import drive
drive.mount('/content/drive')
pd.set_option('display.max_columns', None)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Dataset Airlines

In [7]:
# Conexión a la base de datos
ruta_db = "/content/drive/MyDrive/Colab Notebooks/Datasets/travel.sqlite"
# Conexión a la base de datos
conn = sqlite3.connect(ruta_db)

query = "SELECT * FROM sqlite_master WHERE type='table';"
df_tablas = pd.read_sql(query, conn)
df_tablas

,type,name,tbl_name,rootpage,sql
0,table,aircrafts_data,aircrafts_data,2,CREATE TABLE aircrafts_data (\r\n aircraft_...
1,table,airports_data,airports_data,3,CREATE TABLE airports_data (\r\n airport_co...
2,table,boarding_passes,boarding_passes,4,CREATE TABLE boarding_passes (\r\n ticket_n...
3,table,bookings,bookings,5,CREATE TABLE bookings (\r\n book_ref charac...
4,table,flights,flights,6,CREATE TABLE flights (\r\n flight_id intege...
5,table,seats,seats,7,CREATE TABLE seats (\r\n aircraft_code char...
6,table,ticket_flights,ticket_flights,8,CREATE TABLE ticket_flights (\r\n ticket_no...
7,table,tickets,tickets,9,CREATE TABLE tickets (\r\n ticket_no charac...


In [8]:
# @title ¿Cuántas tablas tiene la base de datos?
query = "SELECT count(*) as 'Cantidad de Tablas' from sqlite_master WHERE type='table'"
cant_tablas = pd.read_sql(query, conn)
cant_tablas

,Cantidad de Tablas
0,8


In [9]:
# @title ¿Qué información hay en cada tabla?

query = "SELECT * from sqlite_master WHERE type='table'"
tablas = pd.read_sql(query, conn)

for tabla in tablas['name']:
  print('Tabla: ', tabla)
  df = pd.read_sql(f"SELECT * FROM {tabla}", conn)
  print(df.head(1))
  print(''.center(100, '-'))

Tabla:  aircrafts_data
  aircraft_code                                            model  range
0           773  {"en": "Boeing 777-300", "ru": "Боинг 777-300"}  11100
----------------------------------------------------------------------------------------------------
Tabla:  airports_data
  airport_code                               airport_name  \
0          YKS  {"en": "Yakutsk Airport", "ru": "Якутск"}   

                                city                            coordinates  \
0  {"en": "Yakutsk", "ru": "Якутск"}  (129.77099609375,62.0932998657226562)   

       timezone  
0  Asia/Yakutsk  
----------------------------------------------------------------------------------------------------
Tabla:  boarding_passes
       ticket_no  flight_id  boarding_no seat_no
0  0005435212351      30625            1      2D
----------------------------------------------------------------------------------------------------
Tabla:  bookings
  book_ref               book_date  total_amount
0 

In [10]:
datos_tabla = pd.read_sql("SELECT * FROM tickets LIMIT(10)", conn)
datos_tabla

,ticket_no,book_ref,passenger_id
0,0005432000987,06B046,8149 604011
1,0005432000988,06B046,8499 420203
2,0005432000989,E170C3,1011 752484
3,0005432000990,E170C3,4849 400049
4,0005432000991,F313DD,6615 976589
5,0005432000992,F313DD,2021 652719
6,0005432000993,F313DD,0817 363231
7,0005432000994,CCC5CB,2883 989356
8,0005432000995,CCC5CB,3097 995546
9,0005432000996,1FB1E4,6866 920231


In [11]:
# @title  1)¿En el dataset de aviones: que tipos de asientos y tiquetes hay?

"""
Para el tipo de asiento
SELECT DISTINCT(fare_conditions) as 'Tipo de Asiento' from seats

SELECT DISTINCT(tf.fare_conditions) as 'Tipo de Ticket'
FROM tickets AS t
INNER JOIN ticket_flights AS tf ON t.ticket_no = tf.ticket_no

"""

query = """
      SELECT DISTINCT(fare_conditions) AS 'Tipo de Asiento' from seats
      """
df_prueba = pd.read_sql(query, conn)
df_prueba

,Tipo de Asiento
0,Business
1,Economy
2,Comfort


In [12]:
subquery = """
        SELECT DISTINCT(tf.fare_conditions)
        FROM tickets AS t
        JOIN ticket_flights AS tf
        ON t.ticket_no = tf.ticket_no
"""
query = f"""
      SELECT DISTINCT(fare_conditions) as 'Tipo Asiento', ({subquery}) AS 'Tipo Ticket'
      FROM seats
"""
prueba3 = pd.read_sql(query, conn)
prueba3

,Tipo Asiento,Tipo Ticket
0,Business,Economy
1,Economy,Economy
2,Comfort,Economy


In [13]:
# @title 2)¿Qué modelos de aviones hay?

query = """
      SELECT model AS 'Modelos de Aviones' FROM aircrafts_data
"""

df_modelos = pd.read_sql(query, conn)
df_modelos

,Modelos de Aviones
0,"{""en"": ""Boeing 777-300"", ""ru"": ""Боинг 777-300""}"
1,"{""en"": ""Boeing 767-300"", ""ru"": ""Боинг 767-300""}"
2,"{""en"": ""Sukhoi Superjet-100"", ""ru"": ""Сухой Суп..."
3,"{""en"": ""Airbus A320-200"", ""ru"": ""Аэробус A320-..."
4,"{""en"": ""Airbus A321-200"", ""ru"": ""Аэробус A321-..."
5,"{""en"": ""Airbus A319-100"", ""ru"": ""Аэробус A319-..."
6,"{""en"": ""Boeing 737-300"", ""ru"": ""Боинг 737-300""}"
7,"{""en"": ""Cessna 208 Caravan"", ""ru"": ""Сессна 208..."
8,"{""en"": ""Bombardier CRJ-200"", ""ru"": ""Бомбардье ..."


In [17]:
 # @title 3)¿Cuántas reservas hay?
query = "SELECT COUNT(total_amount) AS 'Total de Reservas' FROM bookings"
pd.read_sql(query, conn)

,Total de Reservas
0,262788


In [20]:
# @title 4)¿Cuál es el precio medio de las reservas? ¿el más caro? ¿El más barato?
# Tambien trabajarlo con la tabla bookings
query = "SELECT AVG(total_amount) 'Precio AVG', MAX(total_amount) 'Precio MAX', MIN(total_amount) 'Precio MIN' FROM bookings"
df_precios = pd.read_sql(query, conn)
df_precios

,Precio AVG,Precio MAX,Precio MIN
0,79025.605812,1204500,3400


In [23]:
# @title 5)¿Qué rangos de aeronaves hay? (ordenarlos descendentemente)
# Trabajar con la tabla aircrafts_data
query = "SELECT range AS 'Rangos Arenoves' FROM aircrafts_data ORDER BY range DESC"
df_aircraft = pd.read_sql(query, conn)
df_aircraft

,Rangos Arenoves
0,11100
1,7900
2,6700
3,5700
4,5600
5,4200
6,3000
7,2700
8,1200
